<a href="https://colab.research.google.com/github/hydemi/curso-data-science/blob/main/ValidaRDM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U openai langchain-community pypdf langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.3/567.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1


In [2]:
from google.colab import userdata
open_ai_key = userdata.get('open_ai_key')

In [3]:
from openai import OpenAI
client = OpenAI(api_key=open_ai_key)

In [4]:
# Importing necessary modules
import os
#from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
#from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

Loading the document to the notebook

In [ ]:
# Load the file from https://www.bmj.com/content/331/7531/1498.full.pdf to this enviroment
#!wget https://www.bmj.com/content/331/7531/1498.full.pdf

--2024-07-22 03:11:17--  https://www.bmj.com/content/331/7531/1498.full.pdf
Resolving www.bmj.com (www.bmj.com)... 104.18.32.115, 172.64.155.141
Connecting to www.bmj.com (www.bmj.com)|104.18.32.115|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.bmj.com/content/bmj/331/7531/1498.full.pdf [following]
--2024-07-22 03:11:17--  https://www.bmj.com/content/bmj/331/7531/1498.full.pdf
Reusing existing connection to www.bmj.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/pdf]
Saving to: ‘1498.full.pdf.1’

1498.full.pdf.1         [ <=>                ]  92.85K  --.-KB/s    in 0.01s   

2024-07-22 03:11:17 (6.56 MB/s) - ‘1498.full.pdf.1’ saved [95077]



In [17]:
def ask_question(message):
  text = """
    Campo Título >>> Deve informar sucintamente o objetivo da mudança. Não deve constar procedimentos, pacotes e/ou qualquer outra informação que possua campo específico.
    Campo Descrição >>> Deve descrever os objetivos e resultados da mudança, com informações da funcionalidade/rotina/tela/erro que está sendo tratado. Não deve constar procedimentos, pacotes e/ou qualquer outra informação que possua campo específico. Informar ainda os serviços que estão sendo afetadas com a mudança.
      Exemplos:
      Usar: "Correção de subrotina do CFB que estava impedindo a efetivação de PN" ao invés de "Atualizar a subrotina CFBSGC para Produção"
      Usar: "Transferência de elementos visando corrigir a inconsistência na rotina de devolução de cheques pelo motivo 49."; ao invés de "Transferir elemento para produção em atendimento à sustentação 20189999999"
      Usar: "Correção do JOB CTUJDE00 para inclusão do arquivo AOXACL. A alocação dinâmica do arquivo não está funcionando, estamos alterando o JOB para declarar o arquivo." ao invés de "Transferir JOB CTUJDE00"
      Usar: "Atualizar versão do cliente HSM dos servidores SWI3402XX e SWI3402YY. SWI3402XX - Serviços SFC, XYZ / SWI3402YY - Serviços DDA, NPC, ETC"
      Para RDMs em ambiente piloto, além do objetivo da implantação, a descrição deve conter as seguintes informações:
      - Usuários, grupos de usuários ou unidades que participarão;
      - Funcionalidades que serão testadas e avaliadas;
      - Servidores ou rotas que serão utilizados;
      - Por quanto tempo será testado: xx/xx/201x a xx/xx/201x;
      - Horário de funcionamento do piloto: xx:xx às xx:xx;
      - Quem irá testar o piloto (área técnica e negocial) - responsável pela validação do sucesso da execução da estratégia ao final do período;
      - Qual a funcionalidade/ O que será será testado no piloto;
      - Como será desativado (se aplicável).

  """
  # Define a template for the prompt
  prompt_template = f"""Você é um analista de mudanças, especialista em avaliar Requisições de Mudanças de uma instituição.
                    Para validar, você irá considerar o guia de instruções com o que deve ser validado em cada campo.
                    Você receberá perguntas para verificar se o preenchimento de campos específicos da Requisição de Mudança (RDM) está correto conforme o guia de instruções.
                    Você receberá a informação para validação com o campo a que se refere e o conteúdo que foi informado pelo usuário.
                    Você deverá responder o mais fielmente possível às instruções de validação.
                    Caso não atenda aos requisistos do guia de instruções, informe a regra que não foi obedecida e forneça 2 (dois) exemplos de como o campo poderia ser melhor preenchido.
                    Caso não informem a que campo se refere, responda "Você não informou qual campo quer que eu valide".
                    Caso não informem o conteúdo do campo, responda "Você não informou o conteúdo a ser validado".
                    Caso o campo informado não seja encontrado no Guia de instruções, responda "Não tenho conhecimento de como validar este campo"
                No texto abaixo estão as instruções do Guia de Instruções:
                ```{text}```
                """
  #prompt = PromptTemplate.from_template(prompt_template)
  response = client.chat.completions.create(
      messages=[

          {
              "role": "system",
              "content": prompt_template
          },
          {
              "role": "user",
              "content": f"Valide o conteúdo informado de acordo com as instruções: {message}"
          },
      ],
      model="gpt-3.5-turbo",
      temperature=0.7
  )
  return response

In [18]:
response = ask_question("Campo: 'Descrição' / Conteúdo: Atender ao incidente IN23959")
print(response.choices[0].message.content)

Atender ao incidente IN23959 não atende às instruções do guia de instruções. O campo Descrição deve descrever os objetivos e resultados da mudança, com informações da funcionalidade/rotina/tela/erro que está sendo tratado, além de informar os serviços que estão sendo afetados com a mudança. 

Para corrigir, você poderia descrever de forma mais detalhada, por exemplo: "Correção do erro na funcionalidade de cadastro de clientes que impede a inclusão de novos registros, afetando o serviço de atendimento ao cliente". 

Outro exemplo: "Implementação da melhoria na rotina de geração de relatórios para reduzir o tempo de processamento, impactando os serviços de análise de dados".
